In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [8]:
# Configuraciones

#Frecuencias Estándar de DTMF
frecuenciasBajas = [697, 770, 852, 941] # Filas
frecuenciasAltas = [1209, 1336, 1477, 1633] # Columnas

# Mapeo de dígitos a frecuencias DTMF
dtmfMap = {
    (697, 1209): '1', (697, 1336): '2', (697, 1477): '3', (697, 1633): 'A',
    (770, 1209): '4', (770, 1336): '5', (770, 1477): '6', (770, 1633): 'B',
    (852, 1209): '7', (852, 1336): '8', (852, 1477): '9', (852, 1633): 'C',
    (941, 1209): '*', (941, 1336): '0', (941, 1477): '#', (941, 1633): 'D'
}

In [10]:
# Algoritmo de Goertzel
def goertzel(samples, sample_rate, target_freq):
    N = len(samples)
    k = int(0.5 + (N * target_freq) / sample_rate)
    omega = (2.0 * np.pi * k) / N
    coeff = 2.0 * np.cos(omega)

    q1, q2 = 0.0, 0.0

    for sample in samples:
        q0 = coeff * q1 - q2 + sample
        q2 = q1
        q1 = q0
    
    magnitude = q1**2 + q2**2 - coeff * q1 * q2 # No tomo raiz cuadrada porque es costoso, solo quiero comparar más adelante asi que está bien
    return magnitude

In [11]:
def decodificar_dtmf(audio_data, sample_rate, ventana_ms=40, umbral=500):
    """
    Recorre el array del audio y extrae la secuencia de digitos de DTMF
    
    Parámetros:
    audio_data: array numpy con los datos del audio
    sample_rate: frecuencia de muestreo del audio
    ventana_ms: tamaño de la ventana en milisegundos para analizar
    umbral: valor mínimo de magnitud para considerar una frecuencia detectada

    """

    step_size = int(sample_rate * ventana_ms / 1000)
    digitos = []
    ultimo_digito = None # Para evitar repeticiones consecutivas, si se apreta mucho tiempo una tecla
    # Cuando entra a la parte del silencio mas adelante se reinicia, entonces detecta bien numeros consecutivos

    for i in range(0, len(audio_data) - step_size, step_size):
        chunk = audio_data[i:i + step_size]

        # Busco la mejor frecuencia baja, esto significa, la más cercana a umbral, doy robustez por si hay "ruido" al crear el tono
        mejor_baja = None
        max_energia_baja = 0
        for freq in frecuenciasBajas:
            energia = goertzel(chunk, sample_rate, freq)
            if energia > umbral and energia > max_energia_baja:
                max_energia_baja = energia
                mejor_baja = freq
        
        # Busco la mejor frecuencia alta
        mejor_alta = None
        max_energia_alta = 0
        for freq in frecuenciasAltas:
            energia = goertzel(chunk, sample_rate, freq)
            if energia > umbral and energia > max_energia_alta:
                max_energia_alta = energia
                mejor_alta = freq
        
        # Si las encontré deben superar el umbral de ruido para ser considerados, evito falsos positivos, como ruido ambiental
        # Tambien considera que suenen ambas frecuencias, por ejemplo al hablar podria generar una frecuencia de 1209 Hz pero 
        # nada en las bajas, Goertzel devolveria un valor bajo en las altas, y en la baja podria detectar cualquier frecuencia
        # por lo que podria devolver un falso positivo.

        if max_energia_baja > umbral and max_energia_alta > umbral:
            caracter_actual = dtmfMap.get((mejor_baja, mejor_alta))
            if caracter_actual and caracter_actual != ultimo_digito:
                digitos.append(caracter_actual)
                ultimo_digito = caracter_actual
        else:
            ultimo_digito = None
    
    return digitos

In [15]:
tasa, datos = wavfile.read("audios/audio_hablando.wav")
if len(datos.shape) > 1:
    datos = datos[:, 0]  # Usar solo un canal si es estéreo

# Normalizar los datos para que el umbral funcione constante
datos = datos / np.max(np.abs(datos) + 1e-10)  # Evitar división por cero

secuencia_dtmf = decodificar_dtmf(datos, tasa, ventana_ms=40, umbral=50000)

print(f"Secuencia de digitos: {''.join(secuencia_dtmf)}")

Secuencia de digitos: 3876642382
